<a href="https://colab.research.google.com/github/ayub-nur/somali-syllable-and-meter/blob/main/Somali_Syllabifcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title libraries and imports

import pandas
import io
import sys
import re

In [ ]:
#@title consonant and vowel definitions

consonants      = ["'", "b", "t", "j", "x", "kh", "d", "r", "s", "sh", "dh",
                   "c", "g", "f", "q", "k", "l", "m", "n", "w", "h", "y"]

cons_digraph    = ["kh", "sh", "dh"]

# defining all vowel variations
short_vowels    = ["a", "i", "u", "e", "o"]
long_vowels     = ["aa", "ii", "uu", "ee", "oo"]

reg_diphthongs  = ["ay", "aw", "ey", "oy", "ow",]
long_diphthongs = ["aay", "aaw", "eey", "ooy", "oow"]

# defining vowel short hands
diphthongs = reg_diphthongs + long_diphthongs
vowels     = short_vowels + long_vowels

any_vowel  = vowels + diphthongs

In [ ]:
#@title split_into_syllables

def split_into_syllables(word):

    syllables = []
    current_syllable = ""
    i = 0

    index_last_char = len(word)
    # print("last char index", index_last_char)
    while i < len(word):
            # Check if current character is a consonant
            if (i == index_last_char - 1):
                    # print("we in the right section")
                    # print("word is: ", word[i])

                    current_syllable += word[i]
                    # print("k i was: ", i)
                    i += 1
                    # print("l i is: ", i)
            elif word[i:i+2] in consonants:
                current_syllable += word[i:i+2]
                # print("a i was: ", i)
                i += 2
                # print("b i is: ", i)
            elif word[i] in consonants:
                current_syllable += word[i]
                # print("c i was: ", i)
                i += 1
                # print("d i is: ", i)

            # elif word[i] == " ":
            #     current_syllable += word[i]
            #     i += 1




            # Check if current character is a vowel
            if word[i:i+3] in vowels:
            # If the current syllable is not empty
                                            # and the next character is a vowel,

            # add the current syllable to the list and reset it
                if (current_syllable) and (i + 3 < len(word)) and (word[i+3] in vowels):
                    syllables.append(current_syllable)
                    current_syllable = ""
                current_syllable += word[i:i+3]
                # print("e i was: ", i)
                i += 3
                # print("f i is: ", i)
            elif word[i:i+2] in vowels:
                current_syllable += word[i:i+2]
                # print("g i was: ", i)
                i += 2
                # print("h i is: ", i)

            elif (i != index_last_char):
                # print("m i is: ", i)
                # print("index_last_char is: ", index_last_char)

                if (word[i] in vowels):
                    # print("vowel")
                    current_syllable += word[i]
                    # print("i i was: ", i)
                    i += 1
                    # print("j i is: ", i)

          # Add the current syllable to the list and reset
            if current_syllable:
                syllables.append(current_syllable)
                current_syllable = ""

    return syllables


In [ ]:
#@title correct_codas

def correct_codas(syllabified_word):

    counter = 0
    new_word = ""

    for i in syllabified_word:

        #should never happen on zeroth index
        if i in consonants:
            new_word = syllabified_word[counter - 1] + i
            syllabified_word[counter - 1] = new_word

            syllabified_word.remove(i)
            new_word = ""

        counter += 1
    return syllabified_word

In [ ]:
#@title parser

def parser(line):
    line = line.lower()

    regex = re.compile('[^a-zA-Z ]')
    line = regex.sub(' ', line)

    word_arr = line.split()

    parsed_line = []
    for word in word_arr:
        temp_syl = split_into_syllables(word)
        syllables = correct_codas(temp_syl)
        parsed_line += syllables

    return parsed_line

In [ ]:
#@title count_morae
def count_morae(parsed_line):
    morae_list = []
    index = 0
    for syl in parsed_line:
        # # if long_diphthongs in syl:
        if any(s in syl for s in long_diphthongs):
            morae_list.append(2)


        # # if reg_diphthongs in syl:
        elif any(s in syl for s in reg_diphthongs):
            morae_list.append("?")


        # if long_vowels in syl:
        elif any(s in syl for s in long_vowels):
            morae_list.append(2)


        # if short_vowels in syl:
        elif any(s in syl for s in short_vowels):
            # print("index = ", index)
            morae_list.append(1)

    return morae_list


In [ ]:
#@title sum_morae

def sum_morae(morae_list):
    sum = 0
    unknown_morae = 0
    for mora in morae_list:
        if mora == "?":
            unknown_morae += 1
        else:
            sum += mora
    return unknown_morae, sum


In [ ]:
#@title main

line = input("Enter one line: ")

list_of_syllables = parser(line)
corrected_list_of_syllables = correct_codas(list_of_syllables)
print("\n")
print(corrected_list_of_syllables)

list_of_morae = count_morae(corrected_list_of_syllables)
print(list_of_morae)

unknown, known = sum_morae(list_of_morae)

print("\n")
print("known length: ", known)
print("unknown syllables: ", unknown)

Enter one line: fakaanaqaabaqawsayniaw-adnaa


['fa', 'kaa', 'na', 'qaa', 'ba', 'qaw', 'say', 'ni', 'aw', 'ad', 'naa']
[1, 2, 1, 2, 1, '?', '?', 1, '?', 1, 2]


known length:  11
unknown syllables:  3
